In [1]:
import ultralytics

In [2]:
# !unzip "videos.zip" -d "videos"

In [8]:
import cv2
import math
import numpy as np
from ultralytics import YOLO
import argparse

offset = 10
velocityoffset = 10
distancethres = 20
x1 = 0
y1 = 0
drawing = False
lanesCount = [0, 0, 0, 0, 0, 0]
laneSides = {'IN':None, 'OUT':None}
vTypeCount = [0, 0, 0, 0, 0, 0]
vTypeCountOut = [0, 0, 0, 0, 0, 0]
detectionLines = [[666, 492, 1242, 524], [64, 414, 589, 474]]
previousCentersAndIDs = []
id = 0
detectedVehicleIDs = []
cache = []
vehicleVelocities = {} 
import pandas as pd
df = pd.DataFrame(columns=['Video Name',"CAR_IN","CAR_OUT","BUS_IN","BUS_OUT","VAN_IN","VAN_OUT","TRUCK_IN","TRUCK_OUT","BIKE_IN","BIKE_OUT","RICKSHAW_IN","RICKSHAW_OUT","TOTAL_IN","TOTAL_OUT"])

VCount = {'IN':{'car':[],'bus':[],'van':[],'truck':[],'bike':[],'rickshaw':[],'total_count_in':0},
        'OUT':{'car':[],'bus':[],'van':[],'truck':[],'bike':[],'rickshaw':[],'total_count_out':0}}

args={
    'model':'yolov8/runs/detect/train26/weights/best.pt',
    'names':'names.txt',
    'video_path':'videos/',                                                                                                     
    'conf':0.25,
}
class Model:

    def __init__(self):
        self.args = args

        self.model = self.args['model']
    def loadmodel(self):
        self.model = YOLO(self.args['model'])
        return self.model
    
    def names(self):
        my_file = open(self.args['names'], "r")
        data = my_file.read()
        namesTxt = data.split("\n") 
        return namesTxt

class VehicleDetection(Model):
    def __init__(self):
        

        Model.__init__(self)
        self.drawing = False
        self.x = 0
        self.y = 0
        self.framecount = 0
        self.id = 0
        self.model=self.loadmodel()
        self.COLORS = np.random.randint(0, 255, size=(len(self.names()), 3),
        dtype="uint8")
        
    def update_count(self, index, vehicle_class, confidence, detection_row):
        print('update count',index, vehicle_class, confidence, detection_row)
        try:
            if index == 0:
                vTypeCount[self.names().index(vehicle_class)] += 1
            else:
                vTypeCountOut[self.names().index(vehicle_class)] += 1

            detection_type = "IN" if index == 0 else "OUT"
            VCount[detection_type][vehicle_class].append({
                'count': vTypeCount[self.names().index(vehicle_class)] if index == 0 else vTypeCountOut[self.names().index(vehicle_class)],
                'conf': f'{confidence:.2f}',
                'x': detection_row[0],
                'y': detection_row[1],
                'w': detection_row[2],
                'h': detection_row[3]
            })
            laneSides[detection_type] += 1
            VCount[detection_type]['total_count_in' if index == 0 else 'total_count_out'] += 1
#             print(VCount)
        except Exception as e:
            print(f"Error in update_count: {e}")

    
    def run(self,path):
        global VCount
        cap = cv2.VideoCapture(path)
        
        while True:
            centersAndIDs = []
            unavailableIDs = []			
            # boxes = []
            ret,frame = cap.read()
            if not ret:
                break
            (H, W) = frame.shape[:2]
            laneSides["IN"] = lanesCount[0]
            laneSides["OUT"] = lanesCount[1]
            for dl in detectionLines:
                cv2.line(frame, (dl[0], dl[1]), (dl[2], dl[3]), (255, 203, 48), 6)
            
            # if self.framecount % 2 == 0:
            self.framecount+=1
            #     continue

            results = self.model.predict(frame)
            detection = results[0].boxes.data
            # detection=[]
            for ind, row in enumerate(detection):
                confidence=float(row[4])
                obj = int(row[5])
                classes = self.names()[obj]
                if confidence > args['conf']:    
                    (x, y) = (int(row[0]), int(row[1]))
                    (w, h) = (int(row[2]), int(row[3]))
                    center = (x + w) / 2, (y + h) / 2
                    sameVehicleDetected = False
                    alreadyCounted = False
                    for indx,c in enumerate(cache):
                        minDistance = distancethres * ((indx+2)/2)
                        for cid in c:
                            if cid[2] in unavailableIDs:
                                continue
                            distance = math.sqrt((center[0] - cid[0]) ** 2 + (center[1] - cid[1]) ** 2)
                            if distance < minDistance:
                                nearestBox = cid[2]
                                minDistance = distance
                                sameVehicleDetected = True
                        if sameVehicleDetected:
                            break
                    if not sameVehicleDetected:
                        centersAndIDs.append([center[0], center[1],self.id])
                        self.id+=1
                    else:
                        centersAndIDs.append([center[0], center[1], nearestBox])
                        unavailableIDs.append(nearestBox)
            
                    if len(centersAndIDs) !=0:
                        vehicleId = centersAndIDs[len(centersAndIDs) -1][2] 
                    cv2.circle(frame, (int(center[0]), int(center[1])), 4, (41, 18, 252), 5)   # Plot center point

                    for i, dl in enumerate(detectionLines):
                        p1 = np.array([dl[0], dl[1]])
                        p2 = np.array([dl[2], dl[3]])
                        p3 = np.array([center[0], center[1]])
                        if dl[0] < dl[2]:
                            largerX = dl[2]
                            smallerX = dl[0]
                        else:
                            largerX = dl[0]
                            smallerX = dl[2]
                        if dl[1] < dl[3]:
                            largerY = dl[3]
                            smallerY = dl[1]
                        else:
                            largerY = dl[1]
                            smallerY = dl[3]
                    
                        if abs(np.cross(p2 - p1, p3 - p1) / np.linalg.norm(p2 - p1)) < offset and \
                            smallerX - offset < center[0] < largerX + offset and \
                            smallerY - offset < center[1] < largerY + offset:

                            for dvi in detectedVehicleIDs:
                                if dvi == vehicleId:
                                    cv2.line(frame, (dl[0], dl[1]), (dl[2], dl[3]), (90, 224, 63), 6)
                                    alreadyCounted = True
                                    break
                            if not alreadyCounted:
                                detectedVehicleIDs.append(vehicleId)
                                cv2.line(frame, (dl[0], dl[1]), (dl[2], dl[3]), (90, 224, 63), 6)
                                lanesCount[i] += 1
                                try:
                                    self.update_count(i,classes,confidence,row)
                                except Exception as e:
                                    print(f"while loop error: {e}")
                    cv2.rectangle(frame, (x, y), (w, h), (0, 255, 0), 2)  
                    text = "{}: {:.4f}".format(classes, confidence)
                    color = [int(c) for c in self.COLORS[obj]]
                    cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                    cv2.putText(frame, "IN:" + str(laneSides["IN"]), (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (55,18,219), 3)
                    cv2.putText(frame, "OUT:" + str(laneSides["OUT"]), (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (55,18,219), 3)
            
#             cv2.imshow("Frame", frame)
            self.framecount +=1
            cacheSize = 5
            cache.insert(0, centersAndIDs.copy())
            if len(cache) > cacheSize:
                del cache[cacheSize]
            if cv2.waitKey(1)&0xFF==27:
                break
        
        cap.release()
        cv2.destroyAllWindows()
#         print(VCount)
        # update the dataframe
        df.loc[len(df)] = [path.split('/')[-1],len(VCount['IN']['car']),len(VCount['OUT']['car']),len(VCount['IN']['bus']),len(VCount['OUT']['bus'])
        ,len(VCount['IN']['van']),len(VCount['OUT']['van']),len(VCount['IN']['truck']),len(VCount['OUT']['truck']),
        len(VCount['IN']['bike']),len(VCount['OUT']['bike']),len(VCount['IN']['rickshaw']),len(VCount['OUT']['rickshaw']),VCount['IN']['total_count_in'],VCount['OUT']['total_count_out']]
        df.to_csv('vehicle_count.csv',index=False)

        VCount = {'IN':{'car':[],'bus':[],'van':[],'truck':[],'bike':[],'rickshaw':[],'total_count_in':0},
        'OUT':{'car':[],'bus':[],'van':[],'truck':[],'bike':[],'rickshaw':[],'total_count_out':0}}




if __name__ == '__main__':

    # args = parse_args()
    import os
    path = args['video_path']
    for file in os.listdir(path):
        detector = VehicleDetection()
        detector.run(path+file)






0: 384x640 2 cars, 1 van, 2 bikes, 1 rickshaw, 20.4ms
Speed: 1.7ms preprocess, 20.4ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 1 van, 2 bikes, 1 rickshaw, 16.8ms
Speed: 1.3ms preprocess, 16.8ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 1 van, 2 bikes, 1 rickshaw, 18.3ms
Speed: 1.3ms preprocess, 18.3ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 1 van, 1 bike, 18.3ms
Speed: 1.5ms preprocess, 18.3ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 1 van, 1 bike, 1 rickshaw, 18.9ms
Speed: 1.3ms preprocess, 18.9ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 1 van, 2 bikes, 2 rickshaws, 18.9ms
Speed: 1.6ms preprocess, 18.9ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 1 van, 1 bike, 2 rickshaws, 20.5ms
Speed: 1.4ms preprocess, 20.5ms inference, 1.7

update count 1 rickshaw 0.5301227569580078 tensor([3.6423e+02, 3.5764e+02, 5.4606e+02, 5.7954e+02, 5.3012e-01, 5.0000e+00], device='cuda:0')
update count 1 van 0.30701473355293274 tensor([3.6384e+02, 3.5595e+02, 5.4547e+02, 5.8017e+02, 3.0701e-01, 2.0000e+00], device='cuda:0')
update count 0 rickshaw 0.6408699750900269 tensor([1.1147e+03, 4.3824e+02, 1.2426e+03, 5.8620e+02, 6.4087e-01, 5.0000e+00], device='cuda:0')
update count 0 bike 0.6238995790481567 tensor([9.3355e+02, 4.5847e+02, 9.9367e+02, 5.3965e+02, 6.2390e-01, 4.0000e+00], device='cuda:0')
update count 0 bike 0.48415282368659973 tensor([1.0203e+03, 4.6401e+02, 1.0821e+03, 5.5188e+02, 4.8415e-01, 4.0000e+00], device='cuda:0')


0: 384x640 1 car, 2 bikes, 4 rickshaws, 20.4ms
Speed: 1.4ms preprocess, 20.4ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 car, 2 bikes, 4 rickshaws, 18.5ms
Speed: 1.4ms preprocess, 18.5ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 car, 2 bikes, 4 rickshaws, 19.6ms
Speed: 1.3ms preprocess, 19.6ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 car, 2 bikes, 4 rickshaws, 20.5ms
Speed: 1.4ms preprocess, 20.5ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 car, 2 bikes, 4 rickshaws, 19.9ms
Speed: 1.4ms preprocess, 19.9ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 car, 2 bikes, 3 rickshaws, 18.7ms
Speed: 1.4ms preprocess, 18.7ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 car, 1 van, 2 bikes, 4 rickshaws, 17.5ms
Speed: 1.3ms preprocess, 17.5ms inference, 1.5ms postprocess per image at sh

update count 1 rickshaw 0.3745114207267761 tensor([3.7857e+02, 3.4513e+02, 5.5798e+02, 5.5952e+02, 3.7451e-01, 5.0000e+00], device='cuda:0')


0: 384x640 2 cars, 1 van, 2 bikes, 2 rickshaws, 16.9ms
Speed: 1.3ms preprocess, 16.9ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 1 van, 2 bikes, 2 rickshaws, 19.5ms
Speed: 1.4ms preprocess, 19.5ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 1 van, 2 bikes, 2 rickshaws, 16.5ms
Speed: 1.3ms preprocess, 16.5ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 1 van, 3 bikes, 2 rickshaws, 16.2ms
Speed: 1.4ms preprocess, 16.2ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 car, 1 van, 3 bikes, 2 rickshaws, 17.3ms
Speed: 1.4ms preprocess, 17.3ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 car, 1 van, 4 bikes, 2 rickshaws, 16.6ms
Speed: 1.4ms preprocess, 16.6ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 1 van, 2 bikes, 2 rickshaws, 16.4ms
Speed: 1.3ms preprocess, 16.4

...

update count 1 car 0.8128306865692139 tensor([404.7630, 402.7159, 560.6439, 536.7988,   0.8128,   0.0000], device='cuda:0')


0: 384x640 3 cars, 2 rickshaws, 20.4ms
Speed: 1.8ms preprocess, 20.4ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 3 cars, 1 bike, 1 rickshaw, 17.9ms
Speed: 1.6ms preprocess, 17.9ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 3 cars, 1 bike, 1 rickshaw, 18.2ms
Speed: 1.4ms preprocess, 18.2ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 3 cars, 1 bike, 1 rickshaw, 18.9ms
Speed: 1.5ms preprocess, 18.9ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 3 cars, 1 bike, 1 rickshaw, 19.6ms
Speed: 1.5ms preprocess, 19.6ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 3 cars, 3 bikes, 2 rickshaws, 19.0ms
Speed: 1.5ms preprocess, 19.0ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 3 cars, 2 bikes, 1 rickshaw, 19.7ms
Speed: 1.5ms preprocess, 19.7ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 64


0: 384x640 1 car, 2 rickshaws, 18.1ms
Speed: 1.4ms preprocess, 18.1ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 2 rickshaws, 18.2ms
Speed: 1.4ms preprocess, 18.2ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 1 bike, 2 rickshaws, 17.8ms
Speed: 1.4ms preprocess, 17.8ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 3 cars, 1 bike, 2 rickshaws, 19.7ms
Speed: 1.4ms preprocess, 19.7ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 1 bike, 2 rickshaws, 18.2ms
Speed: 1.9ms preprocess, 18.2ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 car, 1 bike, 2 rickshaws, 19.9ms
Speed: 1.5ms preprocess, 19.9ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 car, 1 bike, 2 rickshaws, 18.2ms
Speed: 1.7ms preprocess, 18.2ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 

update count 1 rickshaw 0.8464769124984741 tensor([289.8497, 370.2056, 418.2004, 537.3196,   0.8465,   5.0000], device='cuda:0')


0: 384x640 1 car, 2 bikes, 4 rickshaws, 19.3ms
Speed: 1.5ms preprocess, 19.3ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 car, 2 bikes, 3 rickshaws, 20.7ms
Speed: 1.7ms preprocess, 20.7ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 car, 1 bike, 3 rickshaws, 18.9ms
Speed: 1.6ms preprocess, 18.9ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 car, 2 bikes, 3 rickshaws, 20.0ms
Speed: 1.5ms preprocess, 20.0ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 car, 3 bikes, 4 rickshaws, 18.1ms
Speed: 1.4ms preprocess, 18.1ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 car, 3 bikes, 4 rickshaws, 19.1ms
Speed: 1.4ms preprocess, 19.1ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 car, 3 bikes, 4 rickshaws, 18.1ms
Speed: 1.3ms preprocess, 18.1ms inference, 1.3ms postprocess per image at shape (1, 